In [22]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import os
import json
import jsonlines
import pandas as pd

from collections import defaultdict

In [24]:
def data_add_test(path_en, path_ru, path_alignment, path_mapping):
    
    unique_doc_ids_en = lang_data_parse(path_en)
    unique_doc_ids_ru = lang_data_parse(path_ru)
    
    with open(path_mapping, 'r') as f:
        mapping = f.readlines()
        
    mapping = [x.strip('\n').split('\t') for x in mapping]
    mapping_dict = {}
    
    for el in mapping:
        mapping_dict[el[0]] = el[1]
        
    with open(path_alignment, 'r') as f:
        align = f.readlines()
        
    align = [x.strip('\n').split('\t') for x in align]
    align = [[mapping_dict[x[1]], x[2], x[3]] for x in align]

    return align, unique_doc_ids_en, unique_doc_ids_ru

In [25]:
def lang_data_parse(path):
    
    with open(path, 'r') as f:
        lang = f.readlines()
        
    lang = [x.split('\t') for x in lang]
    
    unique_doc_ids = defaultdict(dict)
    
    for el in lang:
        doc_id = el[0]
        unique_doc_ids[doc_id][el[1]] = el[2].strip('\n')
        
    return unique_doc_ids

In [26]:
path = '/content/texts'

In [27]:
files_en = []
files_fr = []

for el in os.listdir(path):
    if '_fr' in el:
        files_en.append(el)
    else:
        files_fr.append(el)

In [28]:
files_en = sorted(files_en)
files_fr = sorted(files_fr)

In [29]:
results = []

for i, j in zip(files_en, files_fr):
    with open(path + '/' + i, 'r') as f:
        en = f.read()
    with open(path + '/' + j, 'r') as f:
        fr = f.read()
    results.append({"translation": {"fr": fr,
                                    "en": en
                                    }})

In [31]:
align_20, unique_doc_ids_20, unique_doc_ids_fr_20 = data_add_test(
                            '/content/data/medline_fr2en_en.txt',
                            '/content/data/medline_fr2en_fr_21.txt',
                            '/content/data/fr-en_align_validation.tsv',
                            '/content/data/fren_mapping.txt'
                           )

In [32]:
res_20 = []

for el in align_20:
    if 'omitted' not in el:
        try:
            doc_id = el[0]
            sent_en = el[2]
            sent_fr = el[1]
            en = unique_doc_ids_20[doc_id][sent_en]
            fr = unique_doc_ids_fr_20[doc_id][sent_fr]
            res_20.append({"translation": {"fr": fr, "en": en}})
        except KeyError:
            pass

In [34]:
with jsonlines.open('train.jsonl', mode='w') as writer:
    for annotation in results:
        writer.write(annotation)

In [ ]:
import shutil
shutil.rmtree('/content/data')
